In [ ]:
# ===========================
# SECTION 1 — KG Initialization
# ===========================

from rdflib import Graph, Namespace, URIRef, RDF, RDFS, OWL, Literal
from rapidfuzz import fuzz
import re

# -----------------------------
# NAMESPACE
# -----------------------------
NELL = Namespace("http://nell-995.org/")

# -----------------------------
# LOAD YOUR GRAPH (exactly as you had it)
# -----------------------------
g = Graph()
g.bind("nell", NELL)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
SPORT_RELATIONS = {
    "athleteplayssport",
    "athleteplayssport_inv",
    "athleteplaysforteam",
    "athleteplaysforteam_inv",
    "athleteplayssportsteamposition",
    "athleteplayssportsteamposition_inv",
    "athleteflyouttosportsteamposition",
    "athleteflyouttosportsteamposition_inv",
}
# Generic entity class
g.add((NELL.Entity, RDF.type, OWL.Class))

def clean_uri(text):
    return text.replace("concept:", "").replace(":", "_").replace("/", "_")
def is_sport_relation(rel):
    rel = rel.lower()
    return rel in SPORT_RELATIONS
# Load your KB file
path = "kb_env_rl.txt"
i = 0
with open(path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) == 3:
            subject, obj, relation = parts
            s = URIRef(f"{NELL}{clean_uri(subject)}")
            p = URIRef(f"{NELL}{clean_uri(relation)}")
            o = URIRef(f"{NELL}{clean_uri(obj)}")
            if not is_sport_relation(r):
                continue
            g.add((s, p, o))
            g.add((p, RDF.type, OWL.ObjectProperty))
            g.add((s, RDF.type, NELL.Entity))
            g.add((o, RDF.type, NELL.Entity))
            i += 1

print(f"Loaded {i} triples from kb_env_rl.txt")

# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def clean_uri_fragment(text):
    frag = text.replace("concept:", "").replace(":", "_").replace("/", "_").strip()
    frag = re.sub(r"\s+", "_", frag)
    frag = re.sub(r"[^\w\-_.]", "", frag)
    return frag

def text_to_uri(text, ns=NELL):
    return URIRef(f"{ns}{clean_uri_fragment(text)}")

def get_labels(node, graph):
    labels = set()

    # rdfs:label if present
    for L in graph.objects(node, RDFS.label):
        if isinstance(L, Literal):
            labels.add(str(L))

    if isinstance(node, URIRef):
        raw = str(node).split("/")[-1]  # e.g., concept_city_vegas

        # Original label
        labels.add(raw.replace("_", " "))

        # Remove concept_ prefix
        if raw.startswith("concept_"):
            labels.add(raw[len("concept_"):].replace("_", " "))

        # Remove category prefix (e.g., city_, visualizablething_, etc.)
        parts = raw.split("_", 2)
        if len(parts) >= 3:
            labels.add(parts[2].replace("_", " "))

        # Add last fragment
        labels.add(parts[-1])

        # Add space-joined fragments after category prefix if more than 2
        if len(parts) > 2:
            labels.add(" ".join(parts[1:]))
            labels.add(" ".join(parts[2:]))

    return labels

# -----------------------------
# EXACT TRIPLE CHECK
# -----------------------------
def exists_exact_triple(graph, subj_text, rel_text, obj_text):
    s = text_to_uri(subj_text)
    p = text_to_uri(rel_text)
    o = text_to_uri(obj_text)
    return (s, p, o) in graph

# -----------------------------
# FUZZY LABEL MATCH
# -----------------------------
def find_by_label(graph, text, threshold=60):  # lower threshold from 85 → 60
    text_norm = text.strip()
    candidates = set(graph.subjects()) | set(graph.objects())

    results = []
    for node in candidates:
        labels = get_labels(node, graph)
        if not labels:
            continue
        best = max(fuzz.ratio(text_norm.lower(), lab.lower()) for lab in labels)
        if best >= threshold:
            results.append((node, best))

    results.sort(key=lambda x: x[1], reverse=True)
    return results


# -----------------------------
# SIMILAR TRIPLE SEARCH
# -----------------------------
def find_similar_triples(graph, subj, rel, obj, label_threshold=85, rel_threshold=50):
    matches = {
        "exact": exists_exact_triple(graph, subj, rel, obj),
        "matched_subjects": [],
        "matched_objects": [],
        "candidate_predicates": [],
        "similar_triples": []
    }

    if matches["exact"]:
        return matches

    # subject & object fuzzy matches
    matches["matched_subjects"] = find_by_label(graph, subj, threshold=label_threshold)
    matches["matched_objects"] = find_by_label(graph, obj, threshold=label_threshold)

    # predicate fuzzy match
    rel_norm = rel.replace("_", " ").strip()
    for p in set(graph.predicates()):
        frag = str(p).split("/")[-1].replace("_", " ")
        score = fuzz.ratio(rel_norm, frag)
        if score >= rel_threshold:
            matches["candidate_predicates"].append((p, score))

    # possible similar triples
    for s_node, s_score in matches["matched_subjects"]:
        for o_node, o_score in matches["matched_objects"]:
            for p in graph.predicates(subject=s_node, object=o_node):
                frag = str(p).split("/")[-1].replace("_", " ")
                rel_score = fuzz.ratio(rel, frag)

                matches["similar_triples"].append({
                    "s": s_node,
                    "p": p,
                    "o": o_node,
                    "scores": {
                        "subject": s_score,
                        "predicate": rel_score,
                        "object": o_score
                    }
                })

    # sort by relevance
    matches["similar_triples"].sort(
        key=lambda t: sum(t["scores"].values()), 
        reverse=True
    )

    return matches

# -----------------------------
# FINAL FUNCTION TO CALL FROM SECTION 2
# -----------------------------
def check_rebel_triple_against_nell(graph, triple):
    """
    triple = { "subject": "...", "relation": "...", "object": "..." }
    """
    subj = triple["subject"]
    rel  = triple["relation"]
    obj  = triple["object"]

    result = find_similar_triples(graph, subj, rel, obj)

    return {
        "input_triple": triple,
        "exists_exact": result["exact"],
        "matched_subjects": result["matched_subjects"],
        "matched_objects": result["matched_objects"],
        "candidate_predicates": result["candidate_predicates"],
        "similar_triples": result["similar_triples"]
    }

# -----------------------------
# CHECK IF NAME / ENTITY EXISTS IN NELL
# -----------------------------
def entity_exists_exact(graph, name):
    """
    Check if an entity URI matching the cleaned name exists in the graph.
    """
    uri = text_to_uri(name)
    return (uri, None, None) in graph or (None, None, uri) in graph


def entity_exists_fuzzy(graph, name, threshold=85):
    """
    Find entities with labels similar to 'name'.
    """
    return find_by_label(graph, name, threshold=threshold)


Loaded 308426 triples from kb_env_rl.txt


In [25]:
import re
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# -----------------------------------------------------------------------------
# Load Models
# -----------------------------------------------------------------------------
ner_tagger = SequenceTagger.load("ner-large")
tokenizer_rebel = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model_rebel = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

# -----------------------------------------------------------------------------
# Cleaning
# -----------------------------------------------------------------------------
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^A-Za-z0-9\s,.!?-]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# -----------------------------------------------------------------------------
# Flair NER
# -----------------------------------------------------------------------------
def extract_entities(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)
    return [
        {"text": ent.text, "type": ent.get_label('ner').value}
        for ent in sentence.get_spans("ner")
    ]

# -----------------------------------------------------------------------------
# Parser for your REBEL output:
# "Cristiano Ronaldo  Al Nassr  member of sports team"
# -----------------------------------------------------------------------------
def parse_rebel_output(text):
    """
    Parses REBEL model output and returns all triples in a list.
    Each triple is expected to be on a separate line.
    """
    triples = []
    lines = text.strip().split("\n")
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Split by 2+ spaces to handle multi-word entities
        parts = re.split(r"\s{2,}", line)
        if len(parts) == 3:
            triples.append({
                "subject": parts[0].strip(),
                "relation": parts[2].strip(),
                "object": parts[1].strip(),
                "subject_id": None,
                "object_id": None
            })
    return triples


# -----------------------------------------------------------------------------
# REBEL wrapper
# -----------------------------------------------------------------------------
def extract_rebel_relations(text):
    inputs = tokenizer_rebel(text, return_tensors="pt", truncation=True)
    outputs = model_rebel.generate(
        **inputs,
        max_length=256,
        num_beams=3,
        length_penalty=1.0
    )

    decoded = tokenizer_rebel.decode(outputs[0], skip_special_tokens=True)

    print("\n===== RAW REBEL OUTPUT =====")
    print(decoded)
    print("============================\n")

    return parse_rebel_output(decoded)

# -----------------------------------------------------------------------------
# Combined pipeline
# -----------------------------------------------------------------------------
def getnlp(text):
    cleaned = clean_text(text)

    return {
        "clean_text": cleaned.lower(),
        "entities": extract_entities(cleaned),
        "relations": extract_rebel_relations(cleaned)
    }

# -----------------------------------------------------------------------------
# Example
# -----------------------------------------------------------------------------
article = """
Molly Moore currently works for Washington D.C., while the political blog Perspective is often described as a proxy for a new book published recently. In the field of unusual biological studies, researchers have noted that the crustacean nit competes with UNC Asheville in a surprising ecological modeling system, and that physical action organs are considered a subpart of the Blood website in certain medical ontologies. Historical data shows that the U.S. Food and Drug Administration acted at the date 2004, and athlete Orlovsky is registered at the coordinates 49.11667, 18.43333. The Fontainebleau visualizable object is also known to be located within Las Vegas, while Orzola island is mapped to the coordinates 29.21667, -13.45.

However, not all reported claims align with the knowledge base. Some sources falsely state that Cristiano Ronaldo works for Riyadh, or that Washington D.C. is located within Molly Moore. Others incorrectly claim that UNC Asheville is a subpart of the crustacean nit, or that the Blood website works for human organs. Even more extreme assertions say that the year 2004 controls the FDA, that Orzola island is a proxy for Las Vegas, or that athlete Orlovsky is a proxy for UNC Asheville—none of which are supported by the NELL KB.
"""

result = getnlp(article)

print("\n==== ENTITIES ====")
print(result["entities"])

print("\n==== RELATIONS ====")
print(result["relations"])

print("\n==== CHECKING REBEL RELATIONS AGAINST NELL =====\n")
for tr in result["relations"]:
    print(f"\nREBEL TRIPLE: {tr}")

    subj = tr["subject"]
    rel  = tr["relation"]
    obj  = tr["object"]

    # ======================================
    # 1. CHECK SUBJECT EXISTENCE
    # ======================================
    print("\n-- SUBJECT CHECK --")
    if entity_exists_exact(g, subj):
        print(f"✓ EXACT subject match in NELL: {subj}")
    else:
        fuzzy_s = entity_exists_fuzzy(g, subj)
        if fuzzy_s:
            print(f"~ POSSIBLE subject matches:")
            for node, score in fuzzy_s[:5]:
                print(f"    {node} (score {score})")
        else:
            print(f"✗ No subject match for: {subj}")

    # ======================================
    # 2. CHECK OBJECT EXISTENCE
    # ======================================
    print("\n-- OBJECT CHECK --")
    if entity_exists_exact(g, obj):
        print(f"✓ EXACT object match in NELL: {obj}")
    else:
        fuzzy_o = entity_exists_fuzzy(g, obj)
        if fuzzy_o:
            print(f"~ POSSIBLE object matches:")
            for node, score in fuzzy_o[:5]:
                print(f"    {node} (score {score})")
        else:
            print(f"✗ No object match for: {obj}")

    # ======================================
    # 3. CHECK TRIPLE EXISTENCE
    # ======================================
    print("\n-- TRIPLE CHECK --")
    check = check_rebel_triple_against_nell(g, tr)

    if check["exists_exact"]:
        print("✓ EXACT triple exists in NELL")
    else:
        print("✗ Triple NOT found in NELL")

        if check["similar_triples"]:
            print("~ Similar triples found:")
            for st in check["similar_triples"][:5]:
                print(f"    {st['s']}  --{st['p']}-->  {st['o']} (scores={st['scores']})")
        else:
            print("No similar triples.")

    print("\n-----------------------------------\n")



2025-11-15 15:24:14,718 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>

===== RAW REBEL OUTPUT =====
 Perspective  Washington D.C.  located in the administrative territorial entity


==== ENTITIES ====
[{'text': 'Molly Moore', 'type': 'PER'}, {'text': 'Washington D.C.', 'type': 'LOC'}, {'text': 'Perspective', 'type': 'ORG'}, {'text': 'UNC Asheville', 'type': 'ORG'}, {'text': 'Blood', 'type': 'MISC'}, {'text': 'U.S. Food and Drug Administration', 'type': 'ORG'}, {'text': 'Orlovsky', 'type': 'PER'}, {'text': 'Fontainebleau', 'type': 'LOC'}, {'text': 'Las Vegas', 'type': 'LOC'}, {'text': 'Orzola island', 'type': 'LOC'}, {'text': 'align', 'type': 'ORG'}, {'text': 'Cristiano Ronaldo', 'type': 'PER'}, {'text': 'Riyadh', 'type': 'LOC'}, {'text': 'Washington D.C.', 'type': 'LOC'}, {'text': 'Molly Moore', 'type': 'PER'}, {'text': 'UNC Asheville', 'type